<a href="https://colab.research.google.com/github/eddisonpham/Hierarchical-Scene-Captioner/blob/main/TransNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/AlgoverseMMLMResearch/Cloning/TransNetV2

/content/drive/MyDrive/AlgoverseMMLMResearch/Cloning/TransNetV2


In [3]:
!ls -lah inference

total 17K
-rw------- 1 root root  284 Jul 17 03:26 Dockerfile
-rw------- 1 root root   34 Jul 17 03:26 __init__.py
-rw------- 1 root root 3.9K Jul 17 03:26 README.md
-rw------- 1 root root 7.9K Jul 17 03:26 transnetv2.py
drwx------ 2 root root 4.0K Jul 17 03:26 transnetv2-weights


In [4]:
!mkdir -p /content/drive/MyDrive/output

In [11]:
!apt-get -y update >/dev/null 2>&1
!apt-get -y install ffmpeg >/dev/null 2>&1
!ffmpeg -version | head -n 1

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers


In [14]:
!pip install -q --upgrade ffmpeg-python opencv-python

In [15]:
import importlib, sys
print("ffmpeg-python import ->", bool(importlib.util.find_spec("ffmpeg")))

ffmpeg-python import -> True


In [16]:
!python inference/transnetv2.py \
  --weights "inference/transnetv2-weights" \
  --visualize \
  "/content/drive/MyDrive/sample.mov"

2025-08-10 23:12:31.086126: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754867551.110536    4791 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754867551.117781    4791 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1754867551.135892    4791 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754867551.135947    4791 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754867551.135952    4791 computation_placer.cc:177] computation placer alr

In [18]:
INFDIR = "/content/drive/MyDrive/AlgoverseMMLMResearch/Cloning/TransNetV2/inference"
WEIGHTS = f"{INFDIR}/transnetv2-weights"
VIDEO   = "/content/drive/MyDrive/sample.mov"
FRAMES_CSV = "/content/drive/MyDrive/output/shot_boundaries_frames.csv"
SECS_CSV   = "/content/drive/MyDrive/output/shot_boundaries_seconds.csv"

# 1) make the module importable
import sys, os
sys.path.append(INFDIR) if INFDIR not in sys.path else None

# 2) import the model
from transnetv2 import TransNetV2
import csv, cv2

# 3) run inference
os.makedirs(os.path.dirname(FRAMES_CSV), exist_ok=True)
model = TransNetV2(WEIGHTS)
_, _, all_frames_pred = model.predict_video(VIDEO)
scenes = model.predictions_to_scenes(all_frames_pred)

# 4) save frame indices
with open(FRAMES_CSV, "w", newline="") as f:
    w = csv.writer(f); w.writerow(["start_frame","end_frame"]); w.writerows(scenes)

# 5) also save timestamps (seconds)
cap = cv2.VideoCapture(VIDEO); fps = cap.get(cv2.CAP_PROP_FPS); cap.release()
to_sec = lambda f: f / max(fps, 1e-6)
with open(SECS_CSV, "w", newline="") as f:
    w = csv.writer(f); w.writerow(["start_sec","end_sec"])
    for a, b in scenes: w.writerow([to_sec(a), to_sec(b)])

print(f"Saved {len(scenes)} shots to:\n- {FRAMES_CSV}\n- {SECS_CSV}")


[TransNetV2] Extracting frames from /content/drive/MyDrive/sample.mov
[TransNetV2] Processing video frames 1807/1807
Saved 1 shots to:
- /content/drive/MyDrive/output/shot_boundaries_frames.csv
- /content/drive/MyDrive/output/shot_boundaries_seconds.csv
